In [1]:
import numpy as np , pandas as pd 
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
data = pd.read_csv('content/preprocessed_train.csv')

In [4]:
data

,Unnamed: 0,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,month,hour
0,146446,7,1,2016-02-29 09:00:00,1857.2600,0,0,121074,12.8,NaN,8.9,0.0,1021.9,0.0,0.0,29,2,9
1,146449,9,1,2016-02-29 09:00:00,590.9450,0,6,27000,12.8,NaN,8.9,0.0,1021.9,0.0,0.0,29,2,9
2,146454,13,1,2016-02-29 09:00:00,1224.1000,0,0,99380,12.8,NaN,8.9,0.0,1021.9,0.0,0.0,29,2,9
3,146456,14,1,2016-02-29 09:00:00,1435.1500,0,0,86250,12.8,NaN,8.9,0.0,1021.9,0.0,0.0,29,2,9
4,146458,15,1,2016-02-29 09:00:00,422.1040,0,6,83957,12.8,NaN,8.9,0.0,1021.9,0.0,0.0,29,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13362281,20125600,1400,1,2016-03-24 12:00:00,15.3753,15,4,21168,1.7,NaN,1.7,NaN,1016.8,0.0,0.0,24,3,12
13362282,20125601,1400,1,2016-03-24 13:00:00,25.0848,15,4,21168,2.8,NaN,2.2,NaN,1016.6,320.0,1.5,24,3,13
13362283,20125602,1400,1,2016-03-24 14:00:00,32.3439,15,4,21168,5.6,NaN,4.4,NaN,1015.2,110.0,2.1,24,3,14
13362284,20125603,1400,1,2016-03-24 15:00:00,24.2214,15,4,21168,11.1,NaN,4.4,NaN,1013.9,150.0,5.1,24,3,15


In [9]:
b_data=pd.read_csv('content/building_metadata.csv', usecols=['building_id','floor','area'])

In [10]:
b_data

,building_id,floor,area
0,105,1,1
1,106,1,0
2,107,2,2
3,108,1,1
4,109,2,1
...,...,...,...
350,1051,0,0
351,1052,0,1
352,1054,1,2
353,1055,1,1


In [49]:
class fetch_building_data(BaseEstimator, TransformerMixin):
    
    def __init__(self, building_id=122 , meter=0 , primary_use= 99 ): # 99 referes to none
        
        self.building_id = building_id
        self.meter = meter
        self.primary_use = primary_use
        
        
    def fit(self, x, y=None):
        return self  # nothing else to do
    
    
    def transform(self, df):
        
        df= df.drop(columns=['Unnamed: 0' , 'precip_depth_1_hr' , 'cloud_coverage', 'site_id' , 'square_feet'])
        
        if self.primary_use == 99:
            
            df= df.query(f'building_id=={self.building_id} & meter=={self.meter}')
            df.drop(['building_id', 'meter','primary_use' ], axis=1 , inplace=True )
            
        else:
            
            df= df.query(f'building_id=={self.building_id} & meter=={self.meter} & primary_use =={self.primary_use}')
            df.drop(['building_id', 'meter','primary_use'], axis=1 , inplace=True )
            
            
        df.loc[:, "timestamp"] = pd.to_datetime( df.loc[:, "timestamp"] )
        df.set_index('timestamp', inplace=True)
        return df


In [50]:
# data = pd.read_csv('content/preprocessed_train.csv')
fetcher=fetch_building_data()

df= fetcher.transform( data )


In [51]:
df.head()

,meter_reading,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,day,month,hour
timestamp,,,,,,,,,
2016-01-01 00:00:00,134.3,3.8,2.4,1020.9,240.0,3.1,1,1,0
2016-01-01 01:00:00,267.1,3.7,2.4,1021.6,230.0,2.6,1,1,1
2016-01-01 02:00:00,268.7,2.6,1.9,1021.9,0.0,0.0,1,1,2
2016-01-01 03:00:00,267.7,2.0,1.2,1022.3,170.0,1.5,1,1,3
2016-01-01 04:00:00,267.8,2.3,1.8,1022.7,110.0,1.5,1,1,4


In [56]:
from data_fetcher import Fetcher

df_fetcher = Fetcher()

df= df_fetcher.transform( data )
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,meter_reading,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,day,month,hour
timestamp,,,,,,,,,
2016-01-01 00:00:00,134.3,3.8,2.4,1020.9,240.0,3.1,1,1,0
2016-01-01 01:00:00,267.1,3.7,2.4,1021.6,230.0,2.6,1,1,1
2016-01-01 02:00:00,268.7,2.6,1.9,1021.9,0.0,0.0,1,1,2
2016-01-01 03:00:00,267.7,2.0,1.2,1022.3,170.0,1.5,1,1,3
2016-01-01 04:00:00,267.8,2.3,1.8,1022.7,110.0,1.5,1,1,4
